https://dsgiitr.com/blogs/deepwalk/

In [1]:
import torch
import torch.nn as nn
import random

In [2]:
adj_list = [[1,2,3], [0,2,3], [0, 1, 3], [0, 1, 2], [5, 6], [4,6], [4, 5], [1, 3]]
size_vertex = len(adj_list)  # number of vertices
w=3            # window size
d=2            # embedding size
y=200          # walks per vertex
t=6            # walk length 
lr=0.025       # learning rate

v=[0,1,2,3,4,5,6,7] # labels of available vertices

def RandomWalk(node,t):
    walk = [node]        # Walk starts from this node
    
    for i in range(t-1):
        node = adj_list[node][random.randint(0,len(adj_list[node])-1)]
        walk.append(node)

    return walk


RandomWalk(0,20)    

[0, 1, 2, 1, 3, 2, 0, 3, 0, 3, 1, 2, 3, 0, 3, 2, 3, 2, 3, 1]

In [9]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.phi  = nn.Parameter(torch.rand((size_vertex, d), requires_grad=True))    
        self.phi2 = nn.Parameter(torch.rand((d, size_vertex), requires_grad=True))
        
        
    def forward(self, one_hot):
        hidden = torch.matmul(one_hot, self.phi)
        out    = torch.matmul(hidden, self.phi2)
        return out

model = Model()

#http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
#https://dsgiitr.com/blogs/gcn/
def skip_gram(wvi,  w):
    for j in range(len(wvi)):
        for k in range(max(0,j-w) , min(j+w, len(wvi))):
            #print(j,k)
            #generate one hot vector
            one_hot          = torch.zeros(size_vertex)
            one_hot[wvi[j]]  = 1
            out              = model(one_hot)
            print(wvi[j],out[wvi[k]],one_hot,out)
            loss             = torch.log(torch.sum(torch.exp(out))) - out[wvi[k]]
            loss.backward()
            
            for param in model.parameters():
                param.data.sub_(lr*param.grad)
                param.grad.data.zero_()
        

# for i in range(y):
#     random.shuffle(v)
#     for vi in v:
#         wvi=RandomWalk(vi,t)
#         skip_gram(wvi, w)


skip_gram(RandomWalk(0,t), w)        

0 tensor(1.1925, grad_fn=<SelectBackward>) tensor([1., 0., 0., 0., 0., 0., 0., 0.]) tensor([1.1925, 0.3241, 0.4231, 0.1002, 0.7642, 0.6359, 1.0157, 0.8128],
       grad_fn=<SqueezeBackward3>)
0 tensor(0.0981, grad_fn=<SelectBackward>) tensor([1., 0., 0., 0., 0., 0., 0., 0.]) tensor([1.2318, 0.3209, 0.4222, 0.0981, 0.7635, 0.6337, 1.0193, 0.8104],
       grad_fn=<SqueezeBackward3>)
0 tensor(1.2100, grad_fn=<SelectBackward>) tensor([1., 0., 0., 0., 0., 0., 0., 0.]) tensor([1.2100, 0.3139, 0.4139, 0.1301, 0.7497, 0.6219, 1.0013, 0.7956],
       grad_fn=<SqueezeBackward3>)
3 tensor(0.5082, grad_fn=<SelectBackward>) tensor([0., 0., 0., 1., 0., 0., 0., 0.]) tensor([0.5082, 0.0754, 0.1462, 0.0417, 0.2557, 0.1896, 0.4152, 0.2410],
       grad_fn=<SqueezeBackward3>)
3 tensor(0.0418, grad_fn=<SelectBackward>) tensor([0., 0., 0., 1., 0., 0., 0., 0.]) tensor([0.5268, 0.0750, 0.1487, 0.0418, 0.2601, 0.1918, 0.4258, 0.2439],
       grad_fn=<SqueezeBackward3>)
3 tensor(0.5132, grad_fn=<SelectBackward

In [8]:
print(model.phi)


Parameter containing:
tensor([[ 0.2360,  1.3795],
        [-0.5267,  1.2029],
        [-0.2580,  1.2695],
        [-0.5400,  1.0176],
        [ 1.3432, -0.7419],
        [ 0.9130, -1.1489],
        [ 0.7026, -1.2309],
        [-1.3383,  0.1891]], requires_grad=True)
